In [1]:
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import os
from os import listdir
from os.path import isfile, join
from shutil import copyfile
import sagemaker
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()
role = get_execution_role()

bucket = sagemaker_session.default_bucket()
prefix = "sagemaker/DEMO-pytorch-batch-inference-script"
print("Bucket:\n{}".format(bucket))

Bucket:
sagemaker-ap-southeast-1-797237262327


In [2]:
from torchvision import datasets, transforms

datasets.MNIST(
    "data",
    download=True,
    transform=transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
    ),
)

inputs = sagemaker_session.upload_data(path="data", bucket=bucket, key_prefix=prefix)
print("input spec (in this case, just an S3 path): {}".format(inputs))

0it [00:00, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw


0it [00:00, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw


0it [00:00, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw


0it [00:00, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw
Processing...
Done!
input spec (in this case, just an S3 path): s3://sagemaker-ap-southeast-1-797237262327/sagemaker/DEMO-pytorch-batch-inference-script


In [3]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    entry_point="mnist.py",
    role=role,
    framework_version="1.5.1",
    train_instance_count=2,
    train_instance_type="ml.c4.xlarge",
    hyperparameters={"epochs": 6, "backend": "gloo"},
)

estimator.fit({"training": inputs})

ValueError: framework_version or py_version was None, yet image_uri was also None. Either specify both framework_version and py_version, or specify image_uri.

In [6]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    entry_point="mnist.py",
    role=role,
    framework_version="1.8.0",
    py_version='py3',train_instance_count=2,
    train_instance_type="ml.c4.xlarge",
    hyperparameters={"epochs": 6, "backend": "gloo"},
)

estimator.fit({"training": inputs})

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


2021-05-19 20:35:00 Starting - Starting the training job...
2021-05-19 20:35:24 Starting - Launching requested ML instancesProfilerReport-1621456500: InProgress
......
2021-05-19 20:36:24 Starting - Preparing the instances for training............
2021-05-19 20:38:24 Downloading - Downloading input data
2021-05-19 20:38:24 Training - Downloading the training image.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-05-19 20:38:32,473 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2021-05-19 20:38:32,475 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2021-05-19 20:38:32,486 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-05-19 20:38:38,716 sagemaker_pytorch_container.training INFO     Invoking user training script.
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no


2021-05-19 20:38:44 Training - Training image download completed. Training in progress.Train Epoch: 1 [6400/30000 (21%)] Loss: 2.076616
Train Epoch: 1 [12800/30000 (43%)] Loss: 1.221039
Train Epoch: 1 [12800/30000 (43%)] Loss: 1.056066
Train Epoch: 1 [19200/30000 (64%)] Loss: 0.909021
Train Epoch: 1 [19200/30000 (64%)] Loss: 0.942254
Train Epoch: 1 [25600/30000 (85%)] Loss: 0.668599
Train Epoch: 1 [25600/30000 (85%)] Loss: 0.842839
Test set: Average loss: 0.3221, Accuracy: 9099/10000 (91%)

Test set: Average loss: 0.3221, Accuracy: 9099/10000 (91%)

Train Epoch: 2 [6400/30000 (21%)] Loss: 0.456248
Train Epoch: 2 [6400/30000 (21%)] Loss: 0.524471
Train Epoch: 2 [12800/30000 (43%)] Loss: 0.410049
Train Epoch: 2 [19200/30000 (64%)] Loss: 0.518616
Train Epoch: 2 [12800/30000 (43%)] Loss: 0.529621
Train Epoch: 2 [19200/30000 (64%)] Loss: 0.457903
Train Epoch: 2 [25600/30000 (85%)] Loss: 0.562165
Train Epoch: 2 [25600/30000 (85%)] Loss: 0.463513
Test set: Average loss: 0.1936, Accuracy: 942

Training seconds: 298
Billable seconds: 298


In [7]:
sample_folder = "mnist_sample"

In [8]:
# silence the output of the bash command so that the jupyter notebook will not response slowly
!sh prep_inference_data.sh {sample_folder} > /dev/null

--2021-05-19 21:06:51--  https://github.com/myleott/mnist_png/raw/master/mnist_png.tar.gz
Resolving github.com (github.com)... 52.74.223.119
Connecting to github.com (github.com)|52.74.223.119|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/myleott/mnist_png/master/mnist_png.tar.gz [following]
--2021-05-19 21:06:51--  https://raw.githubusercontent.com/myleott/mnist_png/master/mnist_png.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15683414 (15M) [application/octet-stream]
Saving to: ‘mnist_png.tar.gz’

mnist_png.tar.gz    100%[===================>]  14.96M  77.8MB/s    in 0.2s    

2021-05-19 21:06:52 (77.8 MB/s) - ‘mnist_png.tar.gz’ saved [15683414/15683414]



In [9]:
# upload sample images to s3, it will take around 1~2 minutes
inference_inputs = sagemaker_session.upload_data(path=sample_folder, key_prefix=f"{prefix}/images")
display(inference_inputs)

's3://sagemaker-ap-southeast-1-797237262327/sagemaker/DEMO-pytorch-batch-inference-script/images'

In [10]:
model_artefect_s3_location = estimator.model_data

In [11]:
print(model_artefect_s3_location)

s3://sagemaker-ap-southeast-1-797237262327/pytorch-training-2021-05-19-20-35-00-039/output/model.tar.gz
